#Choosing Model

In [1]:
model_name = 'eFUN'
# model_name = 'eFUN-L'
# model_name = 'eFUN-S'
# model_name = 'eFUN-S+'

# Setting Up Environment And Paths

In [2]:
import os
os.chdir('/content')
code_dir = 'FUN'
checkpoint_path = f'/content/model.pth.tar'
data_path = f'/content/FUN/notebooks/example_data'
os.environ['CHECKPOINT_PATH'] = checkpoint_path
!export | grep "CHECKPOINT_PATH"

declare -x CHECKPOINT_PATH="/content/model.pth.tar"


# Getting Code and Installing Requirements

In [3]:
!git clone https://github.com/kfir99/FUN.git $CODE_DIR
os.chdir(f'./{code_dir}')
!pip install jpeg2dct

Cloning into 'FUN'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 235 (delta 37), reused 221 (delta 27), pack-reused 0
Receiving objects: 100% (235/235), 13.85 MiB | 32.82 MiB/s, done.
Resolving deltas: 100% (37/37), done.
     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for jpeg2dct: filename=jpeg2dct-0.2.4-cp36-cp36m-linux_x86_64.whl size=104583 sha256=58507d19d12a46668cedc81d84c1e15257f571073c8b59d50e36b4c42a089e47
  Stored in directory: /root/.cache/pip/wheels/83/1f/ea/f4a5a93d63af5e4e850466204ab77dab5805a4a950bcd24445
Successfully built jpeg2dct


# Getting Checkpoint

In [4]:
model_dict = {
    'eFUN': ('efun', 'https://drive.google.com/uc?id=17GRexna2lMaqELTK-PYgIlwEUk9Hponp'),
    'eFUN-L': ('efun_l', 'https://drive.google.com/uc?id=1sUs0cDZr9cgMQEsXx2xDDqI0AUMG9oK_'),
    'eFUN-S': ('efun_s', 'https://drive.google.com/uc?id=1Vx1rXQ43C3ofWAMhJHtjI8-qDgveEJw3'),
    'eFUN-S+': ('efun_s_plus', 'https://drive.google.com/uc?id=1m2tkGOAjDuRu9Xj9OAT9E78rTMo6e7w1')
}
model_string, model_url = model_dict[model_name]
os.environ['MODEL_URL'] = model_url
!export | grep MODEL_URL

declare -x MODEL_URL="https://drive.google.com/uc?id=17GRexna2lMaqELTK-PYgIlwEUk9Hponp"


In [5]:
 !wget -O $CHECKPOINT_PATH $MODEL_URL

--2020-10-23 11:59:54--  https://drive.google.com/uc?id=17GRexna2lMaqELTK-PYgIlwEUk9Hponp
Resolving drive.google.com (drive.google.com)... 173.194.210.101, 173.194.210.100, 173.194.210.102, ...
Connecting to drive.google.com (drive.google.com)|173.194.210.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q88q7u6u7tj4qs8npcju62d0rafbu8gb/1603454325000/15576367283503380841/*/17GRexna2lMaqELTK-PYgIlwEUk9Hponp [following]
--2020-10-23 11:59:57--  https://doc-00-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q88q7u6u7tj4qs8npcju62d0rafbu8gb/1603454325000/15576367283503380841/*/17GRexna2lMaqELTK-PYgIlwEUk9Hponp
Resolving doc-00-38-docs.googleusercontent.com (doc-00-38-docs.googleusercontent.com)... 173.194.213.132, 2607:f8b0:400c:c0a::84
Connecting to doc-00-38-docs.googleusercontent.com (doc-00-38-docs.googleusercontent.com)|1

#Creating Model

In [6]:
import timm
from timm.data import *
from timm.utils import *
from timm.models import load_checkpoint
from timm.data import Dataset
import torch.nn as nn

In [7]:
model = timm.create_model(
      model_string,
      num_classes=1000,
      in_chans=3)
load_checkpoint(model=model, checkpoint_path=checkpoint_path, strict=False)
model = model.cuda()

#Loading Data

In [8]:
dataset = Dataset(data_path)
data_config = timm.data.resolve_data_config({}, model=model, verbose=True)

In [9]:
loader = create_loader(
    dataset,
    input_size=(3, 224, 224),
    batch_size=1,
    use_prefetcher=False,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    num_workers=1,
    crop_pct=1.0,
    dct=True,
    pin_memory=False,
    tf_preprocessing=False)

#Basic Inference Loop

In [10]:
criterion = nn.CrossEntropyLoss().cuda()

In [11]:
with torch.no_grad():
  for batch_idx, (input, target) in enumerate(loader):
      target = target.cuda()
      input = input.cuda()

      # compute output
      output = model(input)

      loss = criterion(output, target)